# read data

In [1]:
pip install --user joblib

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from nltools.data import Brain_Data
from nltools.mask import expand_mask
from bids import BIDSLayout, BIDSValidator
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, LeaveOneOut, GridSearchCV, cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import permutation_test_score

# data_dir = '/mnt/chrastil/lab/users/lily/path_direction/localizer_beta_series_all_run_exploration'
# data_dir1 = '/mnt/chrastil/lab/users/lily/path_direction/localizer_beta_series_all_run_exploration/derivatives/nibetaseries_no_atlas'
data_dir = '/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_all_run_exploration'
data_dir1 = '/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_all_run_exploration/derivatives/nibetaseries_no_atlas'
folder = 'func'

file_list = glob.glob(os.path.join(data_dir1, 'sub*'))
file_list.pop(1) # get rid of subject 3 who does not have exploration data
# file_list.pop(1) # get rid of subject 5 who does not have exploration data
# file_list.pop(1) # get rid of subject 14 who does not have exploration data

# create a dataframe to store data
colnames = ['sub', 'thalamus', 'retrosplenial', 'precuneus', 'extrastriate', 'early_visual', 'auditory']
roi_explore = pd.DataFrame(np.zeros((len(file_list), len(colnames))), columns = colnames)
roi_explore_rand = pd.DataFrame(np.zeros((len(file_list), len(colnames))), columns = colnames)

counter = 0
for i in file_list:
    sub_num = str.split(i, '/')[-1]
    
    # save subject number
    input_subnum = str.split(sub_num,'-')[-1]
    roi_explore.iloc[counter]['sub'] = int(input_subnum)
    roi_explore_rand.iloc[counter]['sub'] = int(input_subnum)
    
    # create raw data
    E_file_list = glob.glob(os.path.join(data_dir, 'derivatives', 'nibetaseries_no_atlas',sub_num,folder,'*Ex*_desc-E*.nii.gz'))
    print(E_file_list)
    E = Brain_Data(E_file_list)

    N_file_list = glob.glob(os.path.join(data_dir,  'derivatives', 'nibetaseries_no_atlas',sub_num,folder,'*Ex*_desc-N*.nii.gz'))
    N = Brain_Data(N_file_list)

    W_file_list = glob.glob(os.path.join(data_dir, 'derivatives', 'nibetaseries_no_atlas',sub_num,folder,'*Ex*_desc-W*.nii.gz'))
    W = Brain_Data(W_file_list)

    S_file_list = glob.glob(os.path.join(data_dir, 'derivatives', 'nibetaseries_no_atlas',sub_num,folder,'*Ex*_desc-S*.nii.gz'))
    S = Brain_Data(S_file_list)

    data = E.append(N)
    data = data.append(W)
    data = data.append(S)
    Y = pd.DataFrame(np.hstack([np.zeros(len(E.data)), np.ones(len(N.data)),2*np.ones(len(W.data)), 3*np.ones(len(S.data))]))
    data.Y = Y

    # read subcortical template
    atlas_mni_file = os.path.join(data_dir,
                                  "derivatives",
                                  "data",
                                  "HarvardOxford-sub-maxprob-thr25-1mm.nii.gz")


    mask = Brain_Data(atlas_mni_file)
    mask_x = expand_mask(mask)


    ####################### THALAMUS ##############################
    # index - 1 for all because this is python!!!
    tha = mask_x[[4-1,15-1]].sum() # thalamus

    data_masked = data.apply_mask(tha)

    X = data_masked.data
    Y = data_masked.Y.values.ravel()

    kfold1 = KFold(n_splits = 3, shuffle = True, random_state = 0)
    kfold2 = KFold(n_splits = 10, shuffle = True, random_state = 0)
    param_grid = {'C':[0.001,0.01,0.1,1,10,100], 'gamma':[0.001,0.01,0.1,1,10,100] }

    X_std = MinMaxScaler().fit_transform(X)
    svc = SVC(kernel = "rbf")
    cv_scores = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Y, cv= kfold2, n_jobs = -1)


    classification_accuracy = cv_scores.mean()
    roi_explore.iloc[counter]['thalamus'] = classification_accuracy
#     print("Accuracy of thalamus:{:.3f}".format(classification_accuracy))
                                   

    Z = Y.copy()
    random.shuffle(Z)
    cv_scores2 = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Z, cv= kfold2, n_jobs = -1)


    classification_accuracy2 = cv_scores2.mean()
    roi_explore_rand.iloc[counter]['thalamus'] = classification_accuracy2
#     print("Accuracy of thalamus random:{:.3f}".format(classification_accuracy2))
    

    ####################### RETROSPLENIAL CORTEX ##############################

    # read cortical template
    atlas_mni_file = os.path.join(data_dir,
                                  "derivatives",
                                  "data",
                                  "Schaefer2018_100Parcels_17Networks_order_FSLMNI152_1mm.nii.gz")
    mask = Brain_Data(atlas_mni_file)
    mask_x = expand_mask(mask)

    # f = mask.plot()

    rsc = mask_x[[48-1,96-1]].sum() # retrosplenial cortex

    data_masked = data.apply_mask(rsc)

    X = data_masked.data
    Y = data_masked.Y.values.ravel()

    kfold = KFold(n_splits = 10, shuffle = True, random_state = 0)
    param_grid = {'C':[0.001,0.01,0.1,1,10,100], 'gamma':[0.001,0.01,0.1,1,10,100] }

    X_std = MinMaxScaler().fit_transform(X)
    svc = SVC(kernel = "rbf")
    cv_scores = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Y, cv= kfold2, n_jobs = -1)


    classification_accuracy = cv_scores.mean()
    roi_explore.iloc[counter]['retrosplenial'] = classification_accuracy
#     print("Accuracy of retrosplenial cortex:{:.3f}".format(classification_accuracy))

    Z = Y.copy()
    random.shuffle(Z)
    cv_scores2 = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Z, cv= kfold2, n_jobs = -1)


    classification_accuracy2 = cv_scores2.mean()
    roi_explore_rand.iloc[counter]['retrosplenial'] = classification_accuracy2
#     print("Accuracy of retrosplenial cortex random:{:.3f}".format(classification_accuracy2))

    ####################### PRECUNEUS ##############################
    pcun = mask_x[[35-1,36-1]].sum() # precuneus

    data_masked = data.apply_mask(pcun)

    X = data_masked.data
    Y = data_masked.Y.values.ravel()

    kfold = KFold(n_splits = 10, shuffle = True, random_state = 0)
    param_grid = {'C':[0.001,0.01,0.1,1,10,100], 'gamma':[0.001,0.01,0.1,1,10,100] }

    X_std = MinMaxScaler().fit_transform(X)
    svc = SVC(kernel = "rbf")
    cv_scores = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Y, cv= kfold2, n_jobs = -1)


    classification_accuracy = cv_scores.mean()
    roi_explore.iloc[counter]['precuneus'] = classification_accuracy
#     print("Accuracy of precuneus:{:.3f}".format(classification_accuracy))

    Z = Y.copy()
    random.shuffle(Z)
    cv_scores2 = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Z, cv= kfold2, n_jobs = -1)


    classification_accuracy2 = cv_scores2.mean()
    roi_explore_rand.iloc[counter]['precuneus'] = classification_accuracy2
#     print("Accuracy of precuneus random:{:.3f}".format(classification_accuracy2))

    ####################### EXTRASTRIATE CORTEX ##############################
    ext = mask_x[[1-1,2-1,4-1,51-1,52-1,53-1]].sum() # extrastriate cortex

    data_masked = data.apply_mask(ext)

    X = data_masked.data
    Y = data_masked.Y.values.ravel()

    kfold = KFold(n_splits = 10, shuffle = True, random_state = 0)
    param_grid = {'C':[0.001,0.01,0.1,1,10,100], 'gamma':[0.001,0.01,0.1,1,10,100] }

    X_std = MinMaxScaler().fit_transform(X)
    svc = SVC(kernel = "rbf")
    cv_scores = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Y, cv= kfold2, n_jobs = -1)


    classification_accuracy = cv_scores.mean()
    roi_explore.iloc[counter]['extrastriate'] = classification_accuracy
#     print("Accuracy of extrastriate cortex:{:.3f}".format(classification_accuracy))

    Z = Y.copy()
    random.shuffle(Z)
    cv_scores2 = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Z, cv= kfold2, n_jobs = -1)


    classification_accuracy2 = cv_scores2.mean()
    roi_explore_rand.iloc[counter]['extrastriate'] = classification_accuracy2

#     print("Accuracy of extrastriate cortex random:{:.3f}".format(classification_accuracy2))

    ####################### EARLY VISUAL CORTEX ##############################
    evc = mask_x[[3-1,6-1,54-1]].sum() # striate cortex + peri strical
    data_masked = data.apply_mask(evc)

    X = data_masked.data
    Y = data_masked.Y.values.ravel()

    kfold = KFold(n_splits = 10, shuffle = True, random_state = 0)
    param_grid = {'C':[0.001,0.01,0.1,1,10,100], 'gamma':[0.001,0.01,0.1,1,10,100] }

    X_std = MinMaxScaler().fit_transform(X)
    svc = SVC(kernel = "rbf")
    cv_scores = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Y, cv= kfold2, n_jobs = -1)


    classification_accuracy = cv_scores.mean()
    roi_explore.iloc[counter]['early_visual'] = classification_accuracy
#     print("Accuracy of early visual cortex:{:.3f}".format(classification_accuracy))

    Z = Y.copy()
    random.shuffle(Z)
    cv_scores2 = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Z, cv= kfold2, n_jobs = -1)


    classification_accuracy2 = cv_scores2.mean()
    roi_explore_rand.iloc[counter]['early_visual'] = classification_accuracy2
#     print("Accuracy of early visual cortex random:{:.3f}".format(classification_accuracy2))

    ####################### AUDITORY CORTEX ##############################
    aud = mask_x[[10-1,61-1]].sum() # striate cortex + peri strical
    data_masked = data.apply_mask(aud)

    X = data_masked.data
    Y = data_masked.Y.values.ravel()

    kfold = KFold(n_splits = 10, shuffle = True, random_state = 0)
    param_grid = {'C':[0.001,0.01,0.1,1,10,100], 'gamma':[0.001,0.01,0.1,1,10,100] }

    X_std = MinMaxScaler().fit_transform(X)
    svc = SVC(kernel = "rbf")
    cv_scores = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Y, cv= kfold2, n_jobs = -1)


    classification_accuracy = cv_scores.mean()
    roi_explore.iloc[counter]['auditory'] = classification_accuracy
#     print("Accuracy of auditory cortex:{:.3f}".format(classification_accuracy))

    Z = Y.copy()
    random.shuffle(Z)
    cv_scores2 = cross_val_score(GridSearchCV(svc, param_grid, cv = kfold1, n_jobs = -1), X_std, Z, cv= kfold2, n_jobs = -1)


    classification_accuracy2 = cv_scores2.mean()
    roi_explore_rand.iloc[counter]['auditory'] = classification_accuracy2
#     print("Accuracy of auditory cortex random:{:.3f}".format(classification_accuracy2))
    counter += 1

# save data file
str_name = data_dir + '/decision_point_allo_explore.csv'
roi_explore.to_csv(str_name, index=False)

str_name_rand = data_dir + '/decision_point_allo_explore_rand.csv'
roi_explore_rand.to_csv(str_name_rand, index=False)

['/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_all_run_exploration/derivatives/nibetaseries_no_atlas/sub-002/func/sub-002_task-boldEx_run-1_space-MNI152NLin2009cAsym_desc-E_betaseries.nii.gz', '/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_all_run_exploration/derivatives/nibetaseries_no_atlas/sub-002/func/sub-002_task-boldEx2_run-1_space-MNI152NLin2009cAsym_desc-E_betaseries.nii.gz']
['/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_all_run_exploration/derivatives/nibetaseries_no_atlas/sub-005/func/sub-005_task-boldEx_run-1_space-MNI152NLin2009cAsym_desc-E_betaseries.nii.gz', '/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_all_run_exploration/derivatives/nibetaseries_no_atlas/sub-005/func/sub-005_task-boldEx2_run-1_space-MNI152NLin2009cAsym_desc-E_betaseries.nii.gz']
['/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_a

/home/lily/.local/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


['/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_all_run_exploration/derivatives/nibetaseries_no_atlas/sub-036/func/sub-036_task-boldEx_run-4_space-MNI152NLin2009cAsym_desc-E_betaseries.nii.gz', '/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_all_run_exploration/derivatives/nibetaseries_no_atlas/sub-036/func/sub-036_task-boldEx2_run-1_space-MNI152NLin2009cAsym_desc-E_betaseries.nii.gz']
['/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_all_run_exploration/derivatives/nibetaseries_no_atlas/sub-041/func/sub-041_task-boldEx_run-1_space-MNI152NLin2009cAsym_desc-E_betaseries.nii.gz', '/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_all_run_exploration/derivatives/nibetaseries_no_atlas/sub-041/func/sub-041_task-boldEx2_run-1_space-MNI152NLin2009cAsym_desc-E_betaseries.nii.gz']
['/mnt/chrastil/lab/users/lily/face_direction_decision_point/localizer_beta_series_a